In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### **Data Reading**

In [0]:
df=spark.read.format("parquet")\
    .load("abfss://bronze@databrickssk.dfs.core.windows.net/products")
df.display()

In [0]:
df=df.drop('_rescued_data')


In [0]:
df.createOrReplaceTempView("products")

### **Functions**

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN p_price * 0.90

In [0]:
%sql
select product_id,price,databricks_cata.bronze.discount_func(price) as discounted_price 
From products

In [0]:
df=df.withColumn("discounted_price",expr("databricks_cata.bronze.discount_func(price)"))
df.display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.upper_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
  return p_brand.upper()
$$

In [0]:
df.write.mode("overwrite").format("delta")\
    .save("abfss://silver@databrickssk.dfs.core.windows.net/products")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.products_silver
USING DELTA
LOCATION 'abfss://silver@databrickssk.dfs.core.windows.net/products'